## Config

In [1]:
import os

import numpy as np
import pandas as pd

from tqdm import tqdm
from cv2 import imread

from sklearn.metrics import f1_score

import wandb

wandb.init(project="NeuroWood2022-name", entity="nuclear_foxes_team")

wandb: Currently logged in as: nuclear_foxes_team (use `wandb login --relogin` to force relogin)


In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F, CrossEntropyLoss
from torch.optim import Adam, SGD
from torch.utils.data import TensorDataset, DataLoader, Dataset

import torchvision


torch.manual_seed(7575)
np.random.seed(7575)
torch.cuda.empty_cache()

from config import Config
from source import train, WoodDataset

## Custom functions

In [3]:
def transform(x):
    if (x == 0):
        return x
    elif (x == 1):
        return x
    else:
        return 3


def calculate_accuracy(y_pred, y_true):
    correct = (y_pred.argmax(dim=1) == y_true).float().sum()
    return correct


def calculate_f1_score(y_pred, y_true):
    y_pred = np.argmax(y_pred.detach().cpu().numpy(), axis=1)
    y_true = y_true.detach().cpu().numpy()
    return f1_score(y_true, y_pred, average='macro')

## Data loading

In [4]:
train_dataset_full = WoodDataset()

train_set_size = int(len(train_dataset_full) * 0.8)
valid_set_size = len(train_dataset_full) - train_set_size

train_dataset, val_dataset = torch.utils.data.random_split(train_dataset_full, [train_set_size, valid_set_size])

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, pin_memory=True, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE, pin_memory=True, shuffle=True)

In [6]:
main_model = torchvision.models.efficientnet_b2(pretrained=True)

class ModelFinal(nn.Module):
    def __init__(self):
        super(ModelFinal, self).__init__()
        self.main_model = main_model
        fc = nn.Sequential(
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 3),
            nn.Softmax(dim=1)
        )
        self.main_model.fc = fc
    def forward(self, x):
        return self.main_model.forward(x)

In [7]:
model = ModelFinal()

lr = 1e-3
parameters = {
    'optimizer' : SGD(model.parameters(), lr=lr),
    'criterion' : CrossEntropyLoss(),
    'val_criterion' : CrossEntropyLoss(),
    'val_metric' : calculate_accuracy,
    'n_epochs' : 15,
    'device' : Config.DEVICE,
}

wandb.config.update({
  "Learning_rate": lr,
  "Epochs": parameters['n_epochs'],
  "Batch_size": Config.BATCH_SIZE,
  "Network": "Effnetb2 + SGD"
})

In [8]:
train(model, train_dataloader, val_dataloader=val_dataloader, **parameters)

100%|██████████| 154/154 [01:53<00:00,  1.35it/s]


Epoch 1/15 is finished, train loss is 1.7009213210700393.


100%|██████████| 39/39 [00:24<00:00,  1.60it/s]


Validation for epoch 1/15 is finished.


100%|██████████| 154/154 [01:48<00:00,  1.42it/s]


...

Validation for epoch 14/15 is finished.


100%|██████████| 154/154 [01:53<00:00,  1.36it/s]


Epoch 15/15 is finished, train loss is 0.11207699660590781.


100%|██████████| 39/39 [00:24<00:00,  1.61it/s]

Validation for epoch 15/15 is finished.
